<a href="https://colab.research.google.com/github/mashyko/NLP_BERT_Transformers/blob/master/mask_transformers_juman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

pytorch_transformersを用いた単語検索

以下のwebサイトを参考にしました。
http://kento1109.hatenablog.com/entry/2019/08/21/155810

Whole Word Masking版: Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers.zip を以下のサイトからダウンロードします。
http://nlp.ist.i.kyoto-u.ac.jp/DLcounter/lime.cgi?down=http://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JapaneseBertPretrainedModel/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers.zip&name=Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers.zip

ダウンロードしたフォルダーをMy Drive/data/に配置してください。

In [0]:
# jumanPPのインストール：make install をするので、時間がかかります。
!wget https://github.com/ku-nlp/jumanpp/releases/download/v2.0.0-rc2/jumanpp-2.0.0-rc2.tar.xz
!tar xfv jumanpp-2.0.0-rc2.tar.xz  
%cd jumanpp-2.0.0-rc2
!mkdir bld
%cd bld
!cmake .. -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=/usr/local
!make install -j2


In [6]:
! pip install pyknp

In [7]:
!pip install transformers

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
#import pytorch_transformers
from transformers import BertTokenizer, BertModel,BertConfig,BertForMaskedLM

config = BertConfig.from_json_file('/content/drive/My Drive/data/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers/config.json')
#model = BertModel.from_pretrained('/content/drive/My Drive/data/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers/pytorch_model.bin',config=config)


In [0]:
model = BertForMaskedLM.from_pretrained('/content/drive/My Drive/data/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers/pytorch_model.bin',config=config)


In [11]:
from pyknp import Juman
jumanpp = Juman()

text = "宇宙塵は星から生まれ、宇宙に飛んでいきます。これら小さな塵は最終的にはくっつきあって新たな恒星となったり、惑星や月、隕石になったりします。"
result = jumanpp.analysis(text)
tokenized_text = [mrph.midasi for mrph in result.mrph_list()]

print(tokenized_text)

['宇宙', '塵', 'は', '星', 'から', '生まれ', '、', '宇宙', 'に', '飛んで', 'いき', 'ます', '。', 'これ', 'ら', '小さな', '塵', 'は', '最終', '的に', 'は', 'くっつき', 'あって', '新たな', '恒星', 'と', 'なったり', '、', '惑星', 'や', '月', '、', '隕石', 'に', 'なったり', 'し', 'ます', '。']


In [12]:
tokenized_text.insert(0, '[CLS]')
tokenized_text.append('[SEP]')
masked_index = 25
tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)

['[CLS]', '宇宙', '塵', 'は', '星', 'から', '生まれ', '、', '宇宙', 'に', '飛んで', 'いき', 'ます', '。', 'これ', 'ら', '小さな', '塵', 'は', '最終', '的に', 'は', 'くっつき', 'あって', '新たな', '[MASK]', 'と', 'なったり', '、', '惑星', 'や', '月', '、', '隕石', 'に', 'なったり', 'し', 'ます', '。', '[SEP]']


In [13]:
tokenizer = BertTokenizer("/content/drive/My Drive/data/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers/vocab.txt",
                          do_lower_case=False, do_basic_tokenize=False)

indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor)

tensor([[    2,   808, 18566,     9,  1194,    27,   761,     6,   808,     8,
         18535,  4052,  1953,     7,    73,    94,  1971, 18566,     9,   549,
           104,     9,     1,  1441,  1596,     4,    12, 12042,     6,  1806,
            34,    26,     6, 14933,     8, 12042,    31,  1953,     7,     3]])


In [0]:
model.eval()

In [15]:
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

_, predicted_indexes = torch.topk(predictions[0, masked_index], k=5)
predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_indexes.tolist())
print(predicted_tokens)

['星', '天体', '恒星', '惑星', '地球']


以上